# Decision Tree Classifier
 <a class="anchor" id="1.5"></a>

# Importamos Librerias

In [ ]:
#importamos el dataset
import pandas as pd
URL = "../data/processed/BRFSS2021T.csv"
df_diabetes = pd.read_csv(URL)
df_diabetes

# Model + Stratified KFold

In [ ]:
# Define X and Y
X = df_diabetes.iloc[:,:-1].values
y = df_diabetes.iloc[:, -1].values.reshape(-1,1).ravel()

In [ ]:
#Importing metrics
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
# Importing Kfold
from sklearn.model_selection import StratifiedKFold

In [ ]:
# Creating dictionary for the metrics
SVM_scores  = {'accuracy':[],'precision':[],'recall':[]}

In [ ]:
# setting the number of splits
from sklearn.svm import SVC
skf = StratifiedKFold(n_splits=10, random_state = None)
skf.get_n_splits(X, y)
svm = SVC()

In [ ]:
# dividiendo en train y test
from sklearn.svm import SVC
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Iniciating and saving the metrics
    ModelSVM=svm.fit(X_train, y_train)
    svm_prediction = svm.predict(X_test)
    SVM_scores['precision_scores'].append(precision_score(Y_test,svm_prediction))
    SVM_scores['recall_scores'].append(recall_score(Y_test,svm_prediction))
    SVM_scores['accuracy_scores'].append(accuracy_score(Y_test,svm_prediction))

In [12]:
#Mostramos resultado
from numpy import mean 
print('Overall precision_scores:',"{: .2%}".format(mean(SVM_scores['precision_scores'])))
print('Overall recall_scores:',"{: .2%}".format(mean(SVM_scores['recall_scores'])))
print('Overall accuracy_scores:',"{: .2%}".format(mean(SVM_scores['accuracy_scores'])))

KeyError: 'recall_scores'

# Optuna 

In [13]:
def create_model(trial):
    model_type = trial.suggest_categorical('model_type', ['logistic-regression', 'decision-tree', 'svm'])

    if model_type == 'svm':
        kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
        regularization = trial.suggest_uniform('svm-regularization', 0.01, 10)
        degree = trial.suggest_discrete_uniform('degree', 1, 5, 1)
        model = SVC(kernel=kernel, C=regularization, degree=degree)

    if trial.should_prune():
            raise optuna.TrialPruned()

    return model
def objective(trial):
    model = create_model(trial)
    model.fit(X_train, y_train)
    return model_performance(model)

In [18]:
def model_performance(model, X=X_val, y=y_val):
    """
    Get accuracy score on validation/test data from a trained model
    """
    y_pred = model.predict(X)
    return round(accuracy_score(y_pred, y),3)

NameError: name 'X_val' is not defined

In [16]:
import optuna
study = optuna.create_study(direction='maximize', study_name="Nuevo_Estudio", storage='sqlite:///starter.db')

[I 2022-11-27 20:30:13,826] A new study created in RDB with name: Nuevo_Estudio


In [17]:
best_model = create_model(study.best_trial)
best_model.fit(X_train, y_train)
print("Performance: ", model_performance(best_model))

ValueError: Record does not exist.